In [1]:
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.Random import get_random_bytes

def hybrid_rsa_aes_encryption(data, public_key):
    # Generate a random AES key
    aes_key = get_random_bytes(16)

    # Encrypt the data with AES
    cipher_aes = AES.new(aes_key, AES.MODE_EAX)
    ciphertext, tag = cipher_aes.encrypt_and_digest(data)

    # Encrypt the AES key with RSA
    cipher_rsa = PKCS1_OAEP.new(public_key)
    cipher_key = cipher_rsa.encrypt(aes_key)

    return (ciphertext, tag, cipher_aes.nonce, cipher_key)

def hybrid_rsa_aes_decryption(ciphertext, tag, nonce, cipher_key, private_key):
    # Decrypt the AES key with RSA
    cipher_rsa = PKCS1_OAEP.new(private_key)
    aes_key = cipher_rsa.decrypt(cipher_key)

    # Decrypt the data with AES
    cipher_aes = AES.new(aes_key, AES.MODE_EAX, nonce=nonce)
    data = cipher_aes.decrypt_and_verify(ciphertext, tag)

    return data


In [22]:
import random
import time
import threading
from Crypto.PublicKey import RSA

private_key = RSA.generate(2048)
public_key = private_key.publickey()

class Sensor:
    def __init__(self, sensor_id):
        self.sensor_id = sensor_id

    def generate_message(self):
        timestamp = time.time()
        data = random.randint(1, 100)
        return {'sensor_id': self.sensor_id, 'timestamp': timestamp, 'data': data, 'send_timestamp': timestamp}

    def encrypt_message(self):
        message = self.generate_message()
        serialized_message = str(message).encode('utf-8')
        encrypted_message = hybrid_rsa_aes_encryption(serialized_message, public_key)
        return encrypted_message

class Attacker:
    def __init__(self, attacker_id, servers):
        self.attacker_id = attacker_id
        self.servers = servers

    def intercept_message(self, message, sender):
        # The attacker intercepts the message
        intercepted_message = message
        # The attacker resends the intercepted message to a random server
        target_server = random.choice(self.servers)
        target_server.process_message(intercepted_message, f"Attacker {self.attacker_id}")

class Server:
    def __init__(self, server_id, private_key):
        self.server_id = server_id
        self.private_key = private_key
        self.received_messages = set()
        self.successful_messages = 0
        self.successful_attacks = 0
        self.successful_attacks_dropped = 0
        self.start_time = time.time()
        self.end_time = None
        self.processing_units_used = 0 
        self.processed_messages = 0  

    def process_message(self, message, sender):
        decrypted_message = hybrid_rsa_aes_decryption(*message, self.private_key)
        original_message = decrypted_message.decode('utf-8')
        
        timestamp = eval(original_message)['timestamp']
        if (timestamp, sender) not in self.received_messages:
            self.received_messages.add((timestamp, sender))
            self.successful_messages += 1

            if "Attacker" in sender:
                self.successful_attacks += 1

            if self.server_id == 1 and "Attacker" in sender:
                self.successful_attacks_dropped += 1
            else:
                self.processed_messages += 1
                self.processing_units_used += 0.2
                #print(f"Server {self.server_id} received: {original_message} from {sender}")

    def calculate_throughput(self):
        total_time = self.end_time - self.start_time if self.end_time else time.time() - self.start_time
        return self.processed_messages / total_time


    def calculate_resource_utilization(self):
        return self.processing_units_used
    
def simulate_sensor(sensor, servers, attackers):
    for _ in range(100):
        target_server = random.choice(servers)
        encrypted_message = sensor.encrypt_message()
        #print(f"Sensor {sensor.sensor_id} sending: {encrypted_message}")
        
        # Simulate possible interception by attackers
        if random.random() < 0.4:  # 10% chance of interception
            attacker = random.choice(attackers)
            attacker.intercept_message(encrypted_message, f"Sensor {sensor.sensor_id}")
        else:
            target_server.process_message(encrypted_message, f"Sensor {sensor.sensor_id}")

def main():
    # Create 50 sensors, 2 servers, and 5 attackers
    sensors = [Sensor(sensor_id) for sensor_id in range(1, 51)]
    servers = [Server(server_id, private_key) for server_id in range(1, 3)]
    attackers = [Attacker(attacker_id, servers) for attacker_id in range(1, 6)]

    # Create threads for each sensor
    threads = []
    for sensor in sensors:
        thread = threading.Thread(target=simulate_sensor, args=(sensor, servers, attackers))
        threads.append(thread)
        thread.start()

    #All threads running
    for thread in threads:
        thread.join()

    for server in servers:
        resource_utilization = server.calculate_resource_utilization()
        throughput = server.calculate_throughput()
        

        print("Performance Metrics")
        print(f"Server {server.server_id} - Message Recieved: {server.successful_messages} | Processed Messages: {server.processed_messages}")
        print(f"Server {server.server_id} - Successful Attacks: {server.successful_attacks} | Successful Attacks Dropped by the Server: {server.successful_attacks_dropped}")
        print(f"Server {server.server_id} - Resource Utilization: {resource_utilization:.2f} | Throughput: {throughput:.2f}ms/message")


# Main function call
if __name__ == "__main__":
    main()


Performance Metrics
Server 1 - Message Recieved: 2541 | Processed Messages: 1564
Server 1 - Successful Attacks: 977 | Successful Attacks Dropped by the Server: 977
Server 1 - Resource Utilization: 312.80 | Throughput: 75.28ms/message
Performance Metrics
Server 2 - Message Recieved: 2454 | Processed Messages: 2454
Server 2 - Successful Attacks: 959 | Successful Attacks Dropped by the Server: 0
Server 2 - Resource Utilization: 490.80 | Throughput: 118.11ms/message
